# Type classifier
text goes here

In [1]:
import json
from sklearn.neural_network import MLPClassifier
import numpy as np
import pickle
import utils

In [2]:
# train = load_dataset('datasets/DBpedia/smarttask_dbpedia_train.json')
train = utils.load_dataset('datasets/Wikidata/lcquad2_anstype_wikidata_train.json')
test = utils.load_dataset('datasets/Wikidata/lcquad2_anstype_wikidata_test_gold.json')

In [3]:
X_train, y_train, X_test, y_test = utils.prepare_X_y(train, test)
# train_vectors, test_vectors = utils.extract_features(X_train, X_test, 'literal_vectorizer.sav')

# Predict types

In [4]:
def split_bool_literal_reference(X, y):
    bool_map = {}
    literal_map = {}
    resource_map = {}

    for i in range(len(y)):
        if y[i] == 'boolean':
            bool_map[i] = X[i]
        elif y[i] == 'literal':
            literal_map[i] = X[i]
        elif y[i] == 'resource':
            resource_map[i] = X[i]
            
    return bool_map, literal_map, resource_map

In [5]:
bool_map, literal_map, resource_map = split_bool_literal_reference(X_train, y_train)
test_bool_map, test_literal_map, test_resource_map = split_bool_literal_reference(X_test, y_test)

## Train literal

In [6]:
X_train_literal = list(literal_map.values())
y_train_literal = []
for i in list(literal_map.keys()):
    y_train_literal.append(train[i]['type'][0])
    
X_test_literal = list(test_literal_map.values())
y_test_literal = []
for i in list(test_literal_map.keys()):
    y_test_literal.append(test[i]['type'][0])

In [7]:
train_vectors_literal, test_vectors_literal = utils.extract_features(X_train_literal, X_test_literal, 'literal_vectorizer.sav')

In [8]:
clf = MLPClassifier(random_state=1, max_iter=300)
clf.fit(train_vectors_literal, y_train_literal)
pred_literal = clf.predict(test_vectors_literal)
print("Accuracy:", sum(pred_literal==y_test_literal)/len(pred_literal))
pickle.dump(clf, open('type_literal_classifier.sav', 'wb'))

Accuracy: 0.9357466063348416


## Bundle

In [9]:
type_preds = []
count_literal = 0
for i in range(len(test)):
    
    if i in test_bool_map:
        type_preds.append('boolean')
    if i in test_literal_map:
        type_preds.append(pred_literal[count_literal])
        count_literal += 1
    if i in test_resource_map:
        type_preds.append('placeholder')

In [10]:
y_test_types = []
    
for doc in test:
    y_test_types.append(doc['type'][0])

In [11]:
print("Accuracy:", sum(np.array(type_preds)==np.array(y_test_types))/len(type_preds))

Accuracy: 0.3421570772259899
